So there are bouts of activite at are long with large tails and heads. When we window them we might end up with windows that are just he head or tail. These windows dont really reprisent the action. So here we will look athe the # of wrongly pred windows in the firs 30% vs. the # wrong windows in the whole dataset.

In [51]:
import sys
import os
import json
import pandas as pd
import torch
import numpy as np
import plotly.graph_objs as go
import json


#if we want the tails to be the first 15% and last 15%
percent_head = 0.05
percent_tail = 0.05

path_to_model_def = '/home/kuba/Projects/MedicationTakingData/resmodel' #this is were the .py file is 
path_to_dir_with_model_pt_file = '/home/kuba/Projects/MedicationTakingData/resmodel/res_search_00/res_search_00_7'

#the watch and recoding we willbe evaling
WATCH_DIR = '/home/kuba/Documents/Data/Raw/Listerine/3_final/16'
recording = '2024-03-24_13_10_54'

HERTZ = 100
ACTIVITY_NAME_TO_CLASS_INDEX_MAPPING = {
    'water':0,
    'listerine':1,
}

In [52]:
#this will get the bouts then we will split them up into windows 
def get_bouts(recording_dir):
    all_data = []
    all_labels = []
    all_starts = []
    all_ends = []
    
    label_mapping = [
        ('left', 'water', 0.0),
        ('left', 'listerine', 1.0),
        ('right', 'water', 0.0),
        ('right', 'listerine', 1.0)
    ]
    
    with open(os.path.join(recording_dir, 'labels.json')) as f:
        labels = json.load(f)
        
    acc = pd.read_csv(os.path.join(recording_dir, 'acceleration.csv'), skiprows=1)
    gyro = pd.read_csv(os.path.join(recording_dir, 'gyroscope.csv'), skiprows=1)
    
    #convert the timestap to sec
    acc['timestamp'] = (acc['timestamp'] - acc['timestamp'].iloc[0]) * 1e-9
    gyro['timestamp'] = (gyro['timestamp'] - gyro['timestamp'].iloc[0]) * 1e-9
    
    merged_data = pd.merge_asof(acc, gyro, on='timestamp', 
                                suffixes=('_acc', '_gyro'))
    
    for side, liquid, label_value in label_mapping:
        if side in labels and liquid in labels[side]:
            for bout in labels[side][liquid]:
                start_time = bout['start'] 
                end_time = bout['end']
                # Extract data for this bout
                bout_data = merged_data[(merged_data['timestamp'] >= start_time) & (merged_data['timestamp'] <= end_time)].copy()
                
                start_index = (merged_data["timestamp"] < start_time).sum()
                end_index = start_index + len(bout_data) 

                if len(bout_data) > 0:
                    all_data.append(bout_data)
                    all_labels.append(label_value)
                    all_starts.append(start_index)
                    all_ends.append(end_index)

    return all_data, all_labels, all_starts, all_ends

In [53]:
#here we are given a specific start and stop then we find the window data and add too list 
# we also see if its in the head or tail then add it too that list too
def preprocess_window(df, start, window_size, percent_head, percent_tail):
    window = df.iloc[start:start + window_size]
    X_acc = torch.tensor([window[col].values for col in ['x_acc', 'y_acc', 'z_acc']], dtype=torch.float32)
    X_gyro = torch.tensor([window[col].values for col in ['x_gyro', 'y_gyro', 'z_gyro']], dtype=torch.float32)
    all_windows = torch.cat((X_acc, X_gyro), dim=0).unsqueeze(0)
    
    # Calculate bout length and threshold positions
    bout_length = len(df)
    head_threshold = int(bout_length * (percent_head ))
    tail_threshold = bout_length - int(bout_length * (percent_tail))
    

    # print(bout_length)
    # print(head_threshold)
    # print('tail htresh', tail_threshold)



    # Check if window is within head or tail thresholds
    window_end = start + window_size
    is_in_head = start < head_threshold
    is_in_tail = window_end > tail_threshold
    
    filtered_windows = all_windows if (is_in_head or is_in_tail) else None
    
    return all_windows, filtered_windows

In [54]:
def get_windows(bout, label, window_size, stride, percent_head, percent_tail):
    assert 'timestamp' in bout.columns, "Sensor data must include 'timestamp' column."
    # y_df = pd.DataFrame(label, columns=['labels'])
    # df = pd.concat([bout, y_df], axis=1)
    X = []
    X_percent = []
    y = []
    y_percent = []

    #where just move through all posible window vals then send them over to get proccessed
    #if they are valed we willhave the window dat back
    for i in range(0, len(bout) - window_size + 1, stride):
        x, x_percent = preprocess_window(bout, i, window_size, percent_head, percent_tail) 
        X.append(x)
        y.append(label)
        if x_percent is not None:
            X_percent.append(x_percent)
            y_percent.append(label)

    
    X = torch.cat(X, dim=0) if X else torch.tensor([])
    X_percent = torch.cat(X_percent, dim=0) if X_percent else torch.tensor([])
    return X, X_percent, torch.tensor(y, dtype=torch.float32), torch.tensor(y_percent, dtype=torch.float32)

In [55]:
def read_txt(dir):
    txt_path = os.path.join(dir, f'desc.txt')
    with open(txt_path, 'r') as f:
        content = f.read()
    dic = eval(content)
    return dic['window_size'], dic['stride'] 

In [56]:
def get_num_wrong(model, device, X, y):
    X = X.to(device)
    y = y.to(device)
    with torch.no_grad():
        out = torch.sigmoid(model(X))
        predictions = torch.argmax(out, dim=1)

        return (predictions != y).sum().item()

In [57]:
sys.path.append(path_to_model_def)
head_tail = os.path.split(path_to_dir_with_model_pt_file)
model_path = os.path.join(path_to_dir_with_model_pt_file, f'{head_tail[1]}_bestF1.pth')


#get model and meta data
window_size, stride = read_txt(path_to_dir_with_model_pt_file)
print(model_path)

# load model
model = torch.load(model_path)
model.eval()
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)

recording_dir = f'{WATCH_DIR}/{recording}'


all_bouts, all_labels, all_starts, all_ends = get_bouts(recording_dir)

total_wrong = 0
total_wrong_subset = 0

for bout, label in zip(all_bouts, all_labels):
    X, X_percent, y, y_percent = get_windows(bout=bout, label=label, window_size=window_size, stride=stride, percent_head=percent_head, percent_tail=percent_tail)

    # print('X percent shape is ',X_percent.shape, 'and shape of X is: ',X.shape)
    all_wrong = get_num_wrong(model=model, device=device, X=X, y=y)
    subsset_wrong = get_num_wrong(model=model, device=device, X=X_percent, y=y_percent)

    total_wrong += all_wrong
    total_wrong_subset += subsset_wrong



/home/kuba/Projects/MedicationTakingData/resmodel/res_search_00/res_search_00_7/res_search_00_7_bestF1.pth
Using device: cuda:1


In [58]:
print(total_wrong)
print(total_wrong_subset)

381
132
